<a href="https://colab.research.google.com/github/GrzegorzSzczepanek/deep_learning_fun/blob/main/text_category_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import nltk
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import numpy as np

In [32]:
train_dataset = pd.read_parquet("./dataset/train_sft.parquet")
test_dataset = pd.read_parquet("./dataset/test_sft.parquet")

In [33]:
train_dataset.head()

,prompt,prompt_id,messages,category
0,Please summarize the goals for scientists in t...,627a77298cf96a309aa35a62207c4164e22a66f6db7911...,[{'content': 'Please summarize the goals for s...,Summarize
1,Help write a letter of 100 -200 words to my fu...,7d443ef2cc3e34d9dc6ffcdf748c1d2a9880cd48be9c98...,[{'content': 'Help write a letter of 100 -200 ...,Generation
2,"Write a news style post about a fake event, li...",3c975b349494dea76dbbb9c01a2bb925a248efb8ca0944...,[{'content': 'Write a news style post about a ...,Generation
3,"Write a funny, short story about someone who w...",16d804af359db7823c457b7d82809eddaad9a5ea3c91ef...,"[{'content': 'Write a funny, short story about...",Generation
4,Write a letter to the Editor responding to the...,e9da2fa3a6d496c5a5ee500e58e5477362698aaa08e74c...,[{'content': 'Write a letter to the Editor res...,Rewrite


In [34]:
test_dataset.head()

,prompt,prompt_id,messages,category
0,Aster is a chatbot who answers questions with ...,d6c011ffb1ff8a9abe9bd24caf3f9817454a1f054d5d0e...,[{'content': 'Aster is a chatbot who answers q...,Chat
1,What are 5 things I can do when it's raining i...,7682c850c4e5979cef0ad966b8feb119de2bd323bb4081...,[{'content': 'What are 5 things I can do when ...,Brainstorm
2,Write several rhyming sentences from the persp...,45971bbe382b81644487690d2018bc3aec8c807f1be52a...,[{'content': 'Write several rhyming sentences ...,Generation
3,What other strange and obscure music genres ca...,ccdce774f24ede2e799686276e8b72e8b08a4aebf6ba75...,[{'content': 'What other strange and obscure m...,Brainstorm
4,Write one paragraph about the Tlachihualtepetl...,c1f40447167a99977d745672c0c19cb2739b6f5208647a...,[{'content': 'Write one paragraph about the Tl...,Generation


In [63]:
# nltk.download('punkt')

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, all_words):
    # example of how this works sentence = ["hello", "have", "a", "nice", "day"]
    # words [hello, hi, nice, what, pop, see, bye] -> [1. 0. 1. 0. 0. 0. 0.]
    tokenized_sentence = [stem(word) for word in tokenized_sentence]

    bag = np.zeros(len(all_words), dtype=np.float32)
    for i, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[i] = 1.0

    return bag


a = "Maybe getting bitten by a pig hurts, but it's wounds from filbert who heal longer"
print(a)
a = tokenize(a)
print(a)
a = [stem(x) for x in a]
print(a)

Maybe getting bitten by a pig hurts, but it's wounds from filbert who heal longer
['Maybe', 'getting', 'bitten', 'by', 'a', 'pig', 'hurts', ',', 'but', 'it', "'s", 'wounds', 'from', 'filbert', 'who', 'heal', 'longer']
['mayb', 'get', 'bitten', 'by', 'a', 'pig', 'hurt', ',', 'but', 'it', "'s", 'wound', 'from', 'filbert', 'who', 'heal', 'longer']


In [64]:
test_words = ['hello', 'hi', 'nice', 'what', 'pop', 'see', 'bye']
test_tokenized = ["hello", "have", "a", "nice", "day"]
print(bag_of_words(test_tokenized, test_words))

[1. 0. 1. 0. 0. 0. 0.]


In [45]:
all_words = []
Xy = []

In [46]:
test_dataset.messages[0]

array([{'content': 'Aster is a chatbot who answers questions with rhymes.', 'role': 'system'},
       {'content': 'Where did chocolate originate?', 'role': 'user'},
       {'content': 'Chocolate is 4000 years old/Mexico is where it was first sold', 'role': 'assistant'},
       {'content': 'Where was milk chocolate invented?', 'role': 'user'},
       {'content': 'Switzerland was the first to add milk/To make their chocolate smooth as silk', 'role': 'assistant'},
       {'content': 'What are some good desserts that use chocolate?', 'role': 'user'},
       {'content': 'Pie, tart, cookies, and cake/Chocolate is great to bake', 'role': 'assistant'}],
      dtype=object)

In [47]:
i = 0
for prompt in train_dataset.prompt:
    tokenized_prompt = tokenize(prompt)
    all_words.extend(tokenized_prompt)
    Xy.append((tokenized_prompt, train_dataset.category[i]))
    i += 1

In [51]:
ignore_words = ['?', '!', '.', ',', ';', "'", '"']

In [52]:
Xy[0]

(['Please',
  'summarize',
  'the',
  'goals',
  'for',
  'scientists',
  'in',
  'this',
  'text',
  ':',
  'Within',
  'three',
  'days',
  ',',
  'the',
  'intertwined',
  'cup',
  'nest',
  'of',
  'grasses',
  'was',
  'complete',
  ',',
  'featuring',
  'a',
  'canopy',
  'of',
  'overhanging',
  'grasses',
  'to',
  'conceal',
  'it',
  '.',
  'And',
  'decades',
  'later',
  ',',
  'it',
  'served',
  'as',
  'Rinkert',
  '’',
  's',
  'portal',
  'to',
  'the',
  'past',
  'inside',
  'the',
  'California',
  'Academy',
  'of',
  'Sciences',
  '.',
  'Information',
  'gleaned',
  'from',
  'such',
  'nests',
  ',',
  'woven',
  'long',
  'ago',
  'from',
  'species',
  'in',
  'plant',
  'communities',
  'called',
  'transitional',
  'habitat',
  ',',
  'could',
  'help',
  'restore',
  'the',
  'shoreline',
  'in',
  'the',
  'future',
  '.',
  'Transitional',
  'habitat',
  'has',
  'nearly',
  'disappeared',
  'from',
  'the',
  'San',
  'Francisco',
  'Bay',
  ',',
  'and'

In [53]:
all_words = [stem(word) for word in all_words if word not in ignore_words]

In [55]:
all_words[0]

'pleas'

In [57]:
all_words = sorted(set(all_words))

In [60]:
tags = sorted(set(train_dataset.category))
tags

['Brainstorm',
 'Chat',
 'Classify',
 'Closed QA',
 'Coding',
 'Extract',
 'Generation',
 'Open QA',
 'Rewrite',
 'Summarize']

In [65]:
X_train = []
y_train = []

for (category_sentence, category) in Xy:
    bag = bag_of_words(category_sentence, all_words)
    X_train.append(bag)

    label = tags.index(category)
    # this will give index number for every category
    y_train.append(label)

import numpy as np

X_train = np.array(X_train)
y_train = np.array(y_train)

In [66]:
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)

In [67]:
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True
)

# model pipeline

In [125]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_shape, hidden_units, output_shape):
        super(NeuralNetwork, self).__init__()
        self.layer_block = nn.Sequential(
            nn.Linear(input_shape, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, output_shape)
        )

    def forward(self, x):
        return self.layer_block(x)

In [126]:
device = "cuda" if torch.cuda.is_available() else "cpu"
hidden = 8
output = len(tags)
input = len(all_words) # I could use len(X_train[0]) too
model = NeuralNetwork(input, hidden, output).to(device)
print(input, len(all_words))
print(output)

29582 29582
10


In [127]:
model

NeuralNetwork(
  (layer_block): Sequential(
    (0): Linear(in_features=29582, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=10, bias=True)
  )
)

In [128]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
EPOCHS = 1000

In [129]:
def accuracy_fn(y_true: torch.Tensor, y_pred: torch.Tensor) -> [torch.float]:
   # Apply softmax to y_pred
   y_pred = torch.nn.functional.softmax(y_pred, dim=-1)

   # Get the indices of the max log-probability
   pred = torch.argmax(y_pred, dim=1)

   correct = torch.eq(y_true, pred).sum().item()
   acc = (correct / len(y_pred)) * 100
   return acc

In [130]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    model.train()

    for batch in data_loader:
        X, y = batch
        X, y = X.to(device), y.to(device)
        logits = model(X)
        pred = torch.round(torch.sigmoid(logits))

        loss = loss_fn(logits, y)
        acc = accuracy_fn(y_true=y, y_pred=pred)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += acc

    return train_loss / len(data_loader), train_acc / len(data_loader)

In [131]:
from timeit import default_timer as timer
from tqdm.auto import tqdm

train_time_start = timer()

torch.manual_seed(42)

def train(model: torch.nn.Module,
          train_data: torch.utils.data.DataLoader,
        #   test_data: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          accuracy_fn,
          epochs: int,
          device:torch.device = device):

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                        data_loader=train_data,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        accuracy_fn=accuracy_fn,
                                        device=device)
        # test_loss, test_acc = test_step(model=model,
        #                                 data_loader=test_data,
        #                                 loss_fn=loss_fn,
        #                                 accuracy_fn=accuracy_fn,
        #                                 device=device)

        if epoch % 10:
            print(f"\nEpoch: {epoch} | Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.4f}%")

    train_time_end = timer()
    print(f"\nTraining was done in {(train_time_end - train_time_start):.2f} seconds")

In [132]:
train(model=model,
      train_data=train_dataloader,
      accuracy_fn=accuracy_fn,
      loss_fn=loss_function,
      optimizer=optimizer,
      epochs=30)

  0%|          | 0/30 [00:00<?, ?it/s]


Epoch: 1 | Train Loss: 0.938608 | Train Acc: 50.1112%

Epoch: 2 | Train Loss: 0.499597 | Train Acc: 68.2374%

Epoch: 3 | Train Loss: 0.263707 | Train Acc: 79.5349%

Epoch: 4 | Train Loss: 0.145152 | Train Acc: 86.6658%

Epoch: 5 | Train Loss: 0.087750 | Train Acc: 89.1053%

Epoch: 6 | Train Loss: 0.057809 | Train Acc: 90.4642%

Epoch: 7 | Train Loss: 0.039694 | Train Acc: 91.0098%

Epoch: 8 | Train Loss: 0.028651 | Train Acc: 91.6877%

Epoch: 9 | Train Loss: 0.020735 | Train Acc: 91.8546%

Epoch: 11 | Train Loss: 0.011373 | Train Acc: 92.0951%

Epoch: 12 | Train Loss: 0.008834 | Train Acc: 92.4017%

Epoch: 13 | Train Loss: 0.006755 | Train Acc: 92.4949%

Epoch: 14 | Train Loss: 0.005355 | Train Acc: 92.6903%

Epoch: 15 | Train Loss: 0.004215 | Train Acc: 92.5250%

Epoch: 16 | Train Loss: 0.003426 | Train Acc: 92.8196%

Epoch: 17 | Train Loss: 0.002820 | Train Acc: 92.7594%

Epoch: 18 | Train Loss: 0.002354 | Train Acc: 92.7910%

Epoch: 19 | Train Loss: 0.002004 | Train Acc: 92.6738%



In [ ]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True,
                 exist_ok=True
)

MODEL_NAME = "small_language_model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(),
           f=MODEL_SAVE_PATH)

In [ ]:
loaded_model = NeuralNetwork(input, hidden, output).to(device)
loaded_model.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

In [ ]:
example_sentence = "Hello! Tell me what are you doing"
tokenized = tokenize(example_sentence)
X = bag_of_words(tokenized, all_words)
X = torch.from_numpy(X)

output = loaded_model(X)
output = output.unsqueeze(0)
_, predicted = torch.max(output, dim=1)
tag = tags[predicted.item()]

probs = torch.softmax(output, dim=1)
prob = probs[0][predicted.item()]

print(prob)

In [ ]:
def generate_text(model, initial_sentence, max_length):
   sentence = initial_sentence
   while len(sentence) < max_length:
       tokenized = tokenize(sentence)
       X = bag_of_words(tokenized, all_words)
       X = torch.from_numpy(X).unsqueeze(0)
       output = model(X)
       _, predicted = torch.max(output, dim=1)
       next_word = all_words[predicted.item()]
       sentence += ' ' + next_word
   return sentence


In [ ]:
query = "What are you doing?"
response = generate_text(loaded_model, query, max_length=100)
print(response)